In [1]:
import pandas as pd
import requests
import json
import numpy as np

In [2]:
header = {'User-Agent': 'thorntonbill343@gmail.com'}
req = requests.get('https://www.sec.gov/files/company_tickers.json', headers=header)
ciks = req.json()
df = pd.DataFrame(ciks).T
df

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1045810,NVDA,NVIDIA CORP
...,...,...,...
8952,1000209,MBNKP,MEDALLION FINANCIAL CORP
8953,798941,FCNCP,FIRST CITIZENS BANCSHARES INC /DE/
8954,866368,PSBYP,"PS BUSINESS PARKS, INC./MD"
8955,866368,PSBXP,"PS BUSINESS PARKS, INC./MD"


In [3]:
#getting cik number
cikk = None
count = 0
for ticker in df['ticker']:
    count +=1
    if ticker == 'nvda'.upper():
        cikk = str(df.iloc[count-1].cik_str).zfill(10)
cikk

'0001045810'

In [4]:
#creating url
url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK'
full_url = url + cikk + '.json'
full_url
data = requests.get(full_url,headers=header)
data.status_code

200

In [5]:
obj = data.json()

In [6]:
full_url

'https://data.sec.gov/api/xbrl/companyfacts/CIK0001045810.json'

In [6]:
#getting the mean of the revenue
rev_arr = list()
rev_columns = ['start','end','val','form']
rev_df = pd.DataFrame(obj['facts']['us-gaap']['Revenues']['units']['USD'],columns=rev_columns)
rev_df.iloc[0]
counter = 0

for form in rev_df['form'][:]:
    if form == '10-K':
        start_date = pd.to_datetime(rev_df.iloc[counter]['start'])
        end_date = pd.to_datetime(rev_df.iloc[counter]['end'])
        if all([((end_date - start_date).days > 300), rev_df.iloc[counter]['val'] not in rev_arr]):
            #date_vals[df2.iloc[counter]['start']] = df2.iloc[counter]['val']
            rev_arr.append(rev_df.iloc[counter]['val'])
    counter += 1

#revenue mean all years
rev_mean_all_years = round(np.array(rev_arr).mean())

#revenue mean last 5 years
rev_mean_last_five_years = round(np.array(rev_arr[-5:]).mean())
rev_mean_last_five_years

18239000000

In [7]:
#getting cost of goods sold mean
cogs_arr = list()
cogs_columns = ['start','end','val','form']
cogs_df = pd.DataFrame(obj['facts']['us-gaap']['CostOfRevenue']['units']['USD'],columns=cogs_columns)
counter = 0

for cogs_form in cogs_df['form'][:]:
    if cogs_form == '10-K':
        cogs_start_date = pd.to_datetime(cogs_df.iloc[counter]['start'])
        cogs_end_date = pd.to_datetime(cogs_df.iloc[counter]['end'])
        if all([(cogs_end_date - cogs_start_date).days > 300, cogs_df.iloc[counter]['val'] not in cogs_arr]):
            cogs_arr.append(cogs_df.iloc[counter]['val'])
    counter += 1
        # print(cogs_df['start'],cogs_df['end'])
cogs_mean_all_years = round(np.array(cogs_arr).mean())
cogs_mean_last_five_years = round(np.array(cogs_arr[-5:]).mean())


In [8]:
#calculating if rev over 40%
fourty_percent_check_all_years = round((cogs_mean_all_years / rev_mean_all_years) * 100, 2)
fourty_percent_check_last_five_years = round(cogs_mean_last_five_years / rev_mean_last_five_years * 100,2)
print('40% 5 yrs',fourty_percent_check_last_five_years)
print('40% all years', fourty_percent_check_all_years)

40% 5 yrs 38.79
40% all years 42.59


In [9]:
#revenue percent growth/decline avg
rev_growth_arr = list()
count = 0
diff = None
cp_rev_arr = rev_arr.copy()
cp_rev_arr.reverse()
for arr_loc in range(0,len(rev_arr) + 1):
    count += 1
    if count == len(rev_arr):
        break
    
    
    # print(arr_loc,count)
    else:
      
        diff = round(((cp_rev_arr[arr_loc] - cp_rev_arr[count]) / cp_rev_arr[count]) * 100, 2)
        rev_growth_arr.append(diff)
mean_growth_total = round(np.array(rev_growth_arr).mean(),2)
mean_growth_last_five = round(np.array(rev_growth_arr[:5]).mean(),2)
print("mean growth total",mean_growth_total)
print('mean growth last five years', mean_growth_last_five)

mean growth total 14.33
mean growth last five years 33.46


In [11]:
#getting sga values list for compairison of gross profit to sga
sga_arr = list()
sga_columns = ['start','end','val','form']
sga_df = pd.DataFrame(obj['facts']['us-gaap']['SellingGeneralAndAdministrativeExpense']['units']['USD'], columns=sga_columns)
count = 0
sga_df.iloc[0]
for form in sga_df['form'][:]:
    if form == '10-K':
        sga_start_date = pd.to_datetime(sga_df.iloc[count]['start'])
        sga_end_date = pd.to_datetime(sga_df.iloc[count]['end'])
        if all([(sga_end_date - sga_start_date).days > 300,
                sga_df.iloc[count]['val'] not in sga_arr]):
            sga_arr.append(sga_df.iloc[count]['val'])
    count += 1

In [27]:
#stabilizing the list length values for zipping and sga gross profit analysis
cp_rev_arr = rev_arr.copy()
cp_cogs_arr = cogs_arr.copy()
cp_sga_arr = sga_arr.copy()
shortest_lst_length = np.min(np.array([len(cp_rev_arr),len(cp_cogs_arr),len(cp_sga_arr)]))
while len(cp_rev_arr) > shortest_lst_length:
    cp_rev_arr.pop(0)
while len(cp_cogs_arr) > shortest_lst_length:
    cp_cogs_arr.pop(0)
while len(cp_sga_arr) > shortest_lst_length:
    cp_sga_arr.pop(0)
cp_rev_arr.reverse()
cp_cogs_arr.reverse()
cp_sga_arr.reverse()

gross_profit = None
sga_percent = None
# sga_percent_arr = list()
# for rev,cogs,sga in zip(cp_rev_arr,cp_cogs_arr,cp_sga_arr):
#     gross_profit = rev - cogs
#     sga_percent = round((sga / gross_profit) * 100,2)
#     sga_percent_arr.append(sga_percent)

#replacing above code
calc = lambda sga,rev,cogs : round(sga/(rev-cogs)*100,2)
sga_percent_arr = list(map(calc, cp_sga_arr,cp_rev_arr,cp_cogs_arr  ))

sga_analysis_prep_all_years = pd.Series(sga_percent_arr)
sga_analysis_prep_last_five_years = pd.Series(sga_percent_arr[:5])
std_all_years = round(sga_analysis_prep_all_years.std(),2)
std_last_five_years = round(sga_analysis_prep_last_five_years.std(),2)
print('SGA percent of gross profit standard deviation for all years is %.2f' %(std_all_years))
print('SGA percent of gross profit standard deviation last five years is %.2f'%(std_last_five_years))

SGA percent of gross profit standard deviation for all years is 5.19
SGA percent of gross profit standard deviation last five years is 2.30
